In [1]:
import traceback
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

In [2]:
filedate = '20220418'

In [3]:
url = 'https://www.rakuten-sec.co.jp/web/market/data/list.html'

In [4]:
driver = webdriver.Chrome("/Users/ricew/anaconda3/pkgs/chromedriver-binary-2.38-0/Library/bin/chromedriver")

In [5]:
driver.get(url)
time.sleep(3)

In [6]:
from bs4 import BeautifulSoup
import requests

In [7]:
# データを取得
c = driver.page_source
# HTMLをもとに、オブジェクトを作る
soup = BeautifulSoup(c)

In [8]:
# 目的の部分を切り出します。
# summary = soup.find("div",{'id':'tFut'})
summary = soup.find("div",{'class':'indx1_1'})

# tableを見つけます。
tables = summary.find_all('table')

In [9]:
# データを格納するためのリストです。
data = []

# テーブルから行をすべて探し出します。
rows = tables[0].find_all('tr')

# 行から、それぞれのcellを取り出して画面に表示しつつ、dataに格納します。
# try～exceptを入れることにより、その間でタグが見つからなくてエラーになっても続ける
for tr in rows:
    try:
        cols = tr.find_all('th')
        # textを探し出します。
        for th in cols:
            text = th.find(text=True) 
            print(text)
            data.append(text)
        for th in cols:
            text = th.find("a").get_text()
            print(text)
            data.append(text)
        cols = tr.find_all('td')
        # textを探し出します。
        for td in cols:
            text = td.find(text=True) 
            print(text)
            data.append(text) 
    except:
        continue

指標
現在値
前日比
前日比率
更新日時


日経225
26,799.71
0.00
0.00%
--


TOPIX
1,880.08
0.00
0.00%
--


JPX日経400
16,941.54
0.00
0.00%
--


東証プライム市場指数
967.05
-8.37
-0.86%
04/18 16:00


東証スタンダード市場指数
970.90
-3.54
-0.36%
04/18 16:00


東証グロース市場指数
949.18
-17.50
-1.81%
04/18 16:00


旧東証市場第一部指数
1,880.08
-16.23
-0.86%
04/18 16:00


東証グロース市場Core指数
936.61
0.00
0.00%
--


マザーズ総合指数
751.61
0.00
0.00%
--


日経平均VI指数
22.72
0.00
0.00%
--


東証REIT指数
1,999.12
0.00
0.00%
--


NYダウ
34,411.69
-39.54
-0.11%
04/18 17:43


NASDAQ
13,332.36
-18.72
-0.14%
04/18 17:15


S&P500指数
4,391.69
-0.90
-0.02%
04/18 17:43


英FT100指数
7,616.38
+35.58
0.47%
04/14


仏CAC40指数
6,589.35
+47.21
0.72%
04/14


独DAX30指数
14,163.85
+87.41
0.62%
04/14


ロシアRTS指数
926.08
-29.38
-3.07%
04/18


ブラジルボベスパ指数
115,687.25
-494.36
-0.43%
04/18 17:18


インドSENSEX
57,166.74
-1172.19
-2.01%
04/18


香港ハンセン指数
21,518.08
0.00
0.00%
--


ハンセンH株指数
7,385.58
0.00
0.00%
--


ハンセンレッドチップ指数
4,115.29
0.00
0.00%
--


上海総合指数
3,195.524
0.000
0.00%
--


上海A株指数
3,349.13
0.00
0.00%
--


上

In [10]:
index_name = []
current_val = []
diff_val = []
diff_ratio = []
update = []
dells = []
i = 0
# 行を保持する変数を用意します。

# pdfの文字列があるcellを見つけだします。
for item in data:
    if  i %6 == 0:
        index_name.append(item) 
        i = i + 1
    elif i %6 == 1:
        current_val.append(item) 
        i = i + 1
    elif i %6 == 2:
        diff_val.append(item) 
        i = i + 1
    elif i %6 == 3:
        diff_ratio.append(item) 
        i = i + 1
    elif i %6 == 4:
        update.append(item) 
        i = i + 1
    elif i %6 == 5:
        dells.append(item) 
        i = i + 1

In [11]:
# それぞれをSeriesにする
index_name = Series(index_name)
current_val = Series(current_val)
diff_val = Series(diff_val)
diff_ratio = Series(diff_ratio)
update = Series(update)

In [12]:
# 連結してDataFrameにします。
df_market1 = pd.concat([index_name,current_val,diff_val,diff_ratio,update],axis=1)

In [13]:
# 0行名をカラム名として指定し、カラム名と指定した行を削除
df_market1.columns = df_market1.iloc[0]
df_market1.drop(df_market1.index[0],inplace=True)

In [14]:
df_market1

,指標,現在値,前日比,前日比率,更新日時
1,日経225,"26,799.71",0.00,0.00%,--
2,TOPIX,"1,880.08",0.00,0.00%,--
3,JPX日経400,"16,941.54",0.00,0.00%,--
4,東証プライム市場指数,967.05,-8.37,-0.86%,04/18 16:00
5,東証スタンダード市場指数,970.90,-3.54,-0.36%,04/18 16:00
6,東証グロース市場指数,949.18,-17.50,-1.81%,04/18 16:00
7,旧東証市場第一部指数,"1,880.08",-16.23,-0.86%,04/18 16:00
8,東証グロース市場Core指数,936.61,0.00,0.00%,--
9,マザーズ総合指数,751.61,0.00,0.00%,--
10,日経平均VI指数,22.72,0.00,0.00%,--


In [15]:
# データを格納するためのリストです。
data = []
rows = []
# テーブルから行をすべて探し出します。
rows = tables[1].find_all('tr')

# 行から、それぞれのcellを取り出して画面に表示しつつ、dataに格納します。
# try～exceptを入れることにより、その間でタグが見つからなくてエラーになっても続ける
for tr in rows:
    try:
        cols = tr.find_all('th')
        # textを探し出します。
        for th in cols:
            text = th.find(text=True) 
            print(text)
            data.append(text)
        for th in cols:
            text = th.find("a").get_text()
            print(text)
            data.append(text)
        cols = tr.find_all('td')
        # textを探し出します。
        for td in cols:
            text = td.find(text=True) 
            print(text)
            data.append(text) 
    except:
        continue

指標
現在値
前日比
前日比率
更新日時


日経225先物（期近）
26,920
+170
0.64%
04/19 06:00


CME日経平均先物
26,930
+85
0.32%
04/18 18:37


SGX日経平均先物
26,935.00
+160.00
0.60%
04/19 07:37


東証マザース先物（期近）
750.0
+3.0
0.40%
04/19 06:00


JPX日経400先物（期近）
17,020
+125
0.74%
04/19 06:00


JPX国債先物
11,716.01
-5.49
-0.05%
04/18


JPX国債先物レバレッジ
13,645.21
-12.80
-0.09%
04/18


JPX国債先物インバース
8,484.78
+3.97
0.05%
04/18


JPX国債先物ダブルインバース
7,156.54
+6.70
0.09%
04/18


台湾加権先物（期近）
0
--
--%
--


FTSE中国50先物（期近）
0
--
--%
--


E-mini S&P 500
4,398.75
+12.00
0.27%
04/18 18:37


E-mini S&P Mid Cap 400
2,626.6
+8.0
0.31%
04/18 18:22


E-mini S&P Small Cap 600
1,279.9
--
--%
04/18


E-mini NASDAQ-100
13,967.5
+59.75
0.43%
04/18 18:37


E-mini Dow
34,387
+74
0.22%
04/18 18:36


E-mini Nikkei225
26,850
--
--%
04/18


SGX Nifty 50 Index Futures
17,253.00
+27.00
0.16%
04/19 04:59


In [16]:
index_name = []
current_val = []
diff_val = []
diff_ratio = []
update = []
dells = []
i = 0

# それぞれの列に値を割り振ります。
for item in data:
    if   i %6 == 0:
        index_name.append(item) 
        i = i + 1
    elif i %6 == 1:
        current_val.append(item) 
        i = i + 1
    elif i %6 == 2:
        diff_val.append(item) 
        i = i + 1
    elif i %6 == 3:
        diff_ratio.append(item) 
        i = i + 1
    elif i %6 == 4:
        update.append(item) 
        i = i + 1
    elif i %6 == 5:
        dells.append(item) 
        i = i + 1

In [17]:
# それぞれをSeriesにする
index_name = Series(index_name)
current_val = Series(current_val)
diff_val = Series(diff_val)
diff_ratio = Series(diff_ratio)
update = Series(update)

In [18]:
# 連結してDataFrameにします。
df_market2 = pd.concat([index_name,current_val,diff_val,diff_ratio,update],axis=1)

In [19]:
# 0行名をカラム名として指定し、カラム名と指定した行を削除
df_market2.columns = df_market2.iloc[0]
df_market2.drop(df_market2.index[0],inplace=True)

In [20]:
df_market2

,指標,現在値,前日比,前日比率,更新日時
1,日経225先物（期近）,"26,920",+170,0.64%,04/19 06:00
2,CME日経平均先物,"26,930",+85,0.32%,04/18 18:37
3,SGX日経平均先物,"26,935.00",+160.00,0.60%,04/19 07:37
4,東証マザース先物（期近）,750.0,+3.0,0.40%,04/19 06:00
5,JPX日経400先物（期近）,"17,020",+125,0.74%,04/19 06:00
6,JPX国債先物,"11,716.01",-5.49,-0.05%,04/18
7,JPX国債先物レバレッジ,"13,645.21",-12.80,-0.09%,04/18
8,JPX国債先物インバース,"8,484.78",+3.97,0.05%,04/18
9,JPX国債先物ダブルインバース,"7,156.54",+6.70,0.09%,04/18
10,台湾加権先物（期近）,0,--,--%,--


In [21]:
# データを格納するためのリストです。
data = []
rows = []
# テーブルから行をすべて探し出します。
rows = tables[2].find_all('tr')

# 行から、それぞれのcellを取り出して画面に表示しつつ、dataに格納します。
# try～exceptを入れることにより、その間でタグが見つからなくてエラーになっても続ける
for tr in rows:
    try:
        cols = tr.find_all('th')
        # textを探し出します。
        for th in cols:
            text = th.find(text=True) 
            print(text)
            data.append(text)
        for th in cols:
            text = th.find("a").get_text()
            print(text)
            data.append(text)
        cols = tr.find_all('td')
        # textを探し出します。
        for td in cols:
            text = td.find(text=True) 
            print(text)
            data.append(text) 
    except:
        continue

指標
買い
売り
前日比（買い）
前日比率（買い）
更新日時


米ドル/円
127.19
127.20
+0.22
0.17%
04/19 08:39


ユーロ/円
137.03
137.13
+0.16
0.12%
04/19 08:39


豪ドル/円
93.47
93.53
+0.20
0.21%
04/19 08:40


NZドル/円
85.56
85.61
+0.06
0.07%
04/19 08:40


カナダドル/円
100.86
100.93
+0.24
0.24%
04/19 08:39


英ポンド/円
165.35
165.42
+0.19
0.12%
04/19 08:39


スイスフラン/円
134.54
134.65
+0.19
0.14%
04/19 08:39


香港ドル/円
16.2157
16.2223
+0.0239
0.15%
04/19 08:39


トルコリラ/円
8.6797
8.6890
+0.0119
0.14%
04/19 08:39


南アランド/円
8.6642
8.6764
+0.0013
0.02%
04/19 08:39


ロシアルーブル/円
1.5207
1.6181
+0.0024
0.16%
04/19 08:39


韓国ウォン/円
0.1030
0.1030
+0.0002
0.19%
04/19 08:39


メキシコペソ/円
6.4144
6.4203
+0.0147
0.23%
04/19 08:39


ブラジルレアル/円
27.323
27.343
+0.048
0.18%
04/19 08:39


インドルピー/円
1.6668
1.6677
+0.0029
0.17%
04/19 08:39


中国人民元/円
19.9608
19.9937
+0.0314
0.16%
04/19 08:39


シンガポールドル/円
93.3426
93.4411
+0.1194
0.13%
04/19 08:39


タイバーツ/円
3.7702
3.7740
+0.0037
0.10%
04/19 08:39


マレーシアリンギット/円
29.8802
29.9130
+0.0470
0.16%
04/19 08:39


インドネシアルピア/円
0.8860
0.8

In [22]:
index_name = []
current_buy = []
current_sel = []
diff_val = []
diff_ratio = []
update = []
dells = []
i = 0

# それぞれの列に値を割り振ります。
for item in data:
    if   i % 7 == 0:
        index_name.append(item)
        i = i + 1
    elif i % 7 == 1:
        current_buy.append(item) 
        i = i + 1
    elif i % 7 == 2:
        current_sel.append(item) 
        i = i + 1
    elif i % 7 == 3:
        diff_val.append(item) 
        i = i + 1
    elif i % 7 == 4:
        diff_ratio.append(item) 
        i = i + 1
    elif i % 7 == 5:
        update.append(item) 
        i = i + 1
    elif i % 7 == 6:
        dells.append(item) 
        i = i + 1

In [23]:
# それぞれをSeriesにする
index_name = Series(index_name)
current_buy = Series(current_buy)
current_sel = Series(current_sel)
diff_val = Series(diff_val)
diff_ratio = Series(diff_ratio)
update = Series(update)

In [24]:
# 連結してDataFrameにします。
df_market3 = pd.concat([index_name,current_buy,current_sel,diff_val,diff_ratio,update],axis=1)

In [25]:
# 0行名をカラム名として指定し、カラム名と指定した行を削除
df_market3.columns = df_market3.iloc[0]
df_market3.drop(df_market3.index[0],inplace=True)

In [26]:
df_market3

,指標,買い,売り,前日比（買い）,前日比率（買い）,更新日時
1,米ドル/円,127.19,127.20,+0.22,0.17%,04/19 08:39
2,ユーロ/円,137.03,137.13,+0.16,0.12%,04/19 08:39
3,豪ドル/円,93.47,93.53,+0.20,0.21%,04/19 08:40
4,NZドル/円,85.56,85.61,+0.06,0.07%,04/19 08:40
5,カナダドル/円,100.86,100.93,+0.24,0.24%,04/19 08:39
6,英ポンド/円,165.35,165.42,+0.19,0.12%,04/19 08:39
7,スイスフラン/円,134.54,134.65,+0.19,0.14%,04/19 08:39
8,香港ドル/円,16.2157,16.2223,+0.0239,0.15%,04/19 08:39
9,トルコリラ/円,8.6797,8.6890,+0.0119,0.14%,04/19 08:39
10,南アランド/円,8.6642,8.6764,+0.0013,0.02%,04/19 08:39


In [27]:
# データを格納するためのリストです。
data = []
rows = []
# テーブルから行をすべて探し出します。
rows = tables[3].find_all('tr')

# 行から、それぞれのcellを取り出して画面に表示しつつ、dataに格納します。
# try～exceptを入れることにより、その間でタグが見つからなくてエラーになっても続ける
for tr in rows:
    try:
        cols = tr.find_all('th')
        # textを探し出します。
        for th in cols:
            text = th.find(text=True) 
            print(text)
            data.append(text)
        for th in cols:
            text = th.find("a").get_text()
            print(text)
            data.append(text)
        cols = tr.find_all('td')
        # textを探し出します。
        for td in cols:
            text = td.find(text=True) 
            print(text)
            data.append(text) 
    except:
        continue

指標
現在値
前日比
前日比率
更新日時


原油（WTI原油先物）
107.59
+0.64
0.60%
04/18 16:59


天然ガス（Henry Hub）先物（NYMEX）
7.84
+0.02
0.26%
04/18 19:35


金（現物 1oz.あたり）
1,976.4100
-2.0867
-0.11%
04/18 19:39


Gold先物（COMEX)
1,982.9
--
--%
04/18


プラチナ先物（NYMEX）
1,015.4
--
--%
04/18 19:21


Silver先物（COMEX)
26.14
--
--%
04/18


Copper先物（COMEX)
4.793
-0.003
-0.06%
04/18 18:26


Corn先物（CBOT）
813.25
--
--%
04/18


Wheat先物（CBOT）
1,120.5
--
--%
04/18


Soybeans先物（CBOT）
1,714.75
--
--%
04/18


In [28]:
index_name = []
current_val = []
diff_val = []
diff_ratio = []
update = []
dells = []
i = 0

# それぞれの列に値を割り振ります。
for item in data:
    if   i %6 == 0:
        index_name.append(item) 
        i = i + 1
    elif i %6 == 1:
        current_val.append(item) 
        i = i + 1
    elif i %6 == 2:
        diff_val.append(item) 
        i = i + 1
    elif i %6 == 3:
        diff_ratio.append(item) 
        i = i + 1
    elif i %6 == 4:
        update.append(item) 
        i = i + 1
    elif i %6 == 5:
        dells.append(item) 
        i = i + 1

In [29]:
# それぞれをSeriesにする
index_name = Series(index_name)
current_val = Series(current_val)
diff_val = Series(diff_val)
diff_ratio = Series(diff_ratio)
update = Series(update)

In [30]:
# 連結してDataFrameにします。
df_market4 = pd.concat([index_name,current_val,diff_val,diff_ratio,update],axis=1)

In [31]:
# 0行名をカラム名として指定し、カラム名と指定した行を削除
df_market4.columns = df_market4.iloc[0]
df_market4.drop(df_market4.index[0],inplace=True)

In [32]:
df_market4

,指標,現在値,前日比,前日比率,更新日時
1,原油（WTI原油先物）,107.59,+0.64,0.60%,04/18 16:59
2,天然ガス（Henry Hub）先物（NYMEX）,7.84,+0.02,0.26%,04/18 19:35
3,金（現物 1oz.あたり）,"1,976.4100",-2.0867,-0.11%,04/18 19:39
4,Gold先物（COMEX),"1,982.9",--,--%,04/18
5,プラチナ先物（NYMEX）,"1,015.4",--,--%,04/18 19:21
6,Silver先物（COMEX),26.14,--,--%,04/18
7,Copper先物（COMEX),4.793,-0.003,-0.06%,04/18 18:26
8,Corn先物（CBOT）,813.25,--,--%,04/18
9,Wheat先物（CBOT）,"1,120.5",--,--%,04/18
10,Soybeans先物（CBOT）,"1,714.75",--,--%,04/18


In [33]:
# データを格納するためのリストです。
data = []
rows = []
# テーブルから行をすべて探し出します。
rows = tables[4].find_all('tr')

# 行から、それぞれのcellを取り出して画面に表示しつつ、dataに格納します。
# try～exceptを入れることにより、その間でタグが見つからなくてエラーになっても続ける
for tr in rows:
    try:
        cols = tr.find_all('th')
        # textを探し出します。
        for th in cols:
            text = th.find(text=True) 
            print(text)
            data.append(text)
        for th in cols:
            text = th.find("a").get_text()
            print(text)
            data.append(text)
        cols = tr.find_all('td')
        # textを探し出します。
        for td in cols:
            text = td.find(text=True) 
            print(text)
            data.append(text) 
    except:
        continue

指標
年利回り
前日比
更新日時


日本国債3年
-0.030
0.000
04/19 04:00


日本国債5年
0.025
0.000
04/18


日本国債10年
0.243
+0.002
04/19 04:00


米国3年国債
2.669
+0.003
04/19 06:05


米国10年国債
2.857
+0.049
04/19 06:05


ドイツ10年国債
0.841
+0.001
04/18


イギリス10年国債
1.884
-0.006
04/18


フランス10年国債
1.336
-0.008
04/18


スペイン10年国債
1.792
+0.004
04/18


ポルトガル10年国債
1.856
+0.005
04/18


ユーロ圏10年国債
0.841
+0.001
04/18


イタリア10年国債
2.523
+0.030
04/19 04:00


ギリシャ10年国債
2.924
+0.036
04/19 06:32


ハンガリー10年国債
6.670
-0.110
04/14


In [34]:
index_name = []
current_val = []
diff_val = []
update = []
dells = []
i = 0

# それぞれの列に値を割り振ります。
for item in data:
    if   i %5 == 0:
        index_name.append(item) 
        i = i + 1
    elif i %5 == 1:
        current_val.append(item) 
        i = i + 1
    elif i %5 == 2:
        diff_val.append(item) 
        i = i + 1
    elif i %5 == 3:
        update.append(item) 
        i = i + 1
    elif i %5 == 4:
        dells.append(item) 
        i = i + 1

In [35]:
# それぞれをSeriesにする
index_name = Series(index_name)
current_val = Series(current_val)
diff_val = Series(diff_val)
update = Series(update)

In [36]:
# 連結してDataFrameにします。
df_market5 = pd.concat([index_name,current_val,diff_val,update],axis=1)

In [37]:
# 0行名をカラム名として指定し、カラム名と指定した行を削除
df_market5.columns = df_market5.iloc[0]
df_market5.drop(df_market5.index[0],inplace=True)

In [38]:
df_market5

,指標,年利回り,前日比,更新日時
1,日本国債3年,-0.030,0.000,04/19 04:00
2,日本国債5年,0.025,0.000,04/18
3,日本国債10年,0.243,+0.002,04/19 04:00
4,米国3年国債,2.669,+0.003,04/19 06:05
5,米国10年国債,2.857,+0.049,04/19 06:05
6,ドイツ10年国債,0.841,+0.001,04/18
7,イギリス10年国債,1.884,-0.006,04/18
8,フランス10年国債,1.336,-0.008,04/18
9,スペイン10年国債,1.792,+0.004,04/18
10,ポルトガル10年国債,1.856,+0.005,04/18


In [39]:
# データを格納するためのリストです。
data = []
rows = []
# テーブルから行をすべて探し出します。
rows = tables[5].find_all('tr')

# 行から、それぞれのcellを取り出して画面に表示しつつ、dataに格納します。
# try～exceptを入れることにより、その間でタグが見つからなくてエラーになっても続ける
for tr in rows:
    try:
        cols = tr.find_all('th')
        # textを探し出します。
        for th in cols:
            text = th.find(text=True) 
            print(text)
            data.append(text)
        for th in cols:
            text = th.find("a").get_text()
            print(text)
            data.append(text)
        cols = tr.find_all('td')
        # textを探し出します。
        for td in cols:
            text = td.find(text=True) 
            print(text)
            data.append(text) 
    except:
        continue

指標
政策金利
更新日時


日本 無担保コール翌日物
-0.019
2022/04/18


日本 公定歩合
0.3
2008/12/19


アメリカ フェデラルファンド金利
0.50
2022/03/16


カナダ 翌日物金利誘導目標
1.00
2022/04/13


ユーロ 市場調整金利
--
2016/03/10


イギリス レポ金利
0.75
2022/03/17


スイス 3ヵ月物LIBOR誘導目標
-0.7500
2022/03/24


南アフリカ レポ金利
4.25
2022/03/24


香港 再割引金利
5.000
2022/04/14


オーストラリア　政策金利
0.10
2022/04/19 09:02


ニュージーランド 目標短期金利
1.50
2022/04/19 09:30


インド レポ金利
4.00
2022/03/02


中国 貸出金利
4.35
2015/10/24


ブラジル Selicターゲットレート
11.75
2022/03/16


ロシア リファイナンス金利
--
--


トルコ 中央銀行ON DEPO
12.50
2022/04/18


メキシコ オーバーナイト金利
6.48
2022/04/18 17:05


In [40]:
index_name = []
current_val = []
update = []
dells = []
i = 0

# それぞれの列に値を割り振ります。
for item in data:
    if   i %4 == 0:
        index_name.append(item) 
        i = i + 1
    elif i %4 == 1:
        current_val.append(item) 
        i = i + 1
    elif i %4 == 2:
        update.append(item) 
        i = i + 1
    elif i %4 == 3:
        dells.append(item) 
        i = i + 1

In [41]:
# それぞれをSeriesにする
index_name = Series(index_name)
current_val = Series(current_val)
update = Series(update)

In [42]:
# 連結してDataFrameにします。
df_market6 = pd.concat([index_name,current_val,update],axis=1)

In [43]:
# 0行名をカラム名として指定し、カラム名と指定した行を削除
df_market6.columns = df_market6.iloc[0]
df_market6.drop(df_market6.index[0],inplace=True)

In [44]:
df_market6

,指標,政策金利,更新日時
1,日本 無担保コール翌日物,-0.019,2022/04/18
2,日本 公定歩合,0.3,2008/12/19
3,アメリカ フェデラルファンド金利,0.50,2022/03/16
4,カナダ 翌日物金利誘導目標,1.00,2022/04/13
5,ユーロ 市場調整金利,--,2016/03/10
6,イギリス レポ金利,0.75,2022/03/17
7,スイス 3ヵ月物LIBOR誘導目標,-0.7500,2022/03/24
8,南アフリカ レポ金利,4.25,2022/03/24
9,香港 再割引金利,5.000,2022/04/14
10,オーストラリア 政策金利,0.10,2022/04/19 09:02


In [45]:
# 行列入れ替え
df_market11 = df_market1.T

In [46]:
# 1行目だけ残して後は削除
df_market12 = df_market11.drop(df_market11.index[[2,3,4]]).reset_index(drop=True)

In [47]:
# 2行目だけ残して後は削除
df_market13 = df_market11.drop(df_market11.index[[1,3,4]]).reset_index(drop=True)

In [48]:
# 3行目だけ残して後は削除
df_market14 = df_market11.drop(df_market11.index[[1,2,4]]).reset_index(drop=True)

In [49]:
df_market15 = pd.concat([df_market12, df_market13, df_market14], axis=1)

In [50]:
df_market15

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,日経225,TOPIX,JPX日経400,東証プライム市場指数,東証スタンダード市場指数,東証グロース市場指数,旧東証市場第一部指数,東証グロース市場Core指数,マザーズ総合指数,日経平均VI指数,...,ハンセンH株指数,ハンセンレッドチップ指数,上海総合指数,上海A株指数,上海・香港ストックコネクト,CSI300,シンガポールST指数,タイSET指数,クアラルンプール総合指数,ジャカルタ総合指数
1,"26,799.71","1,880.08","16,941.54",967.05,970.90,949.18,"1,880.08",936.61,751.61,22.72,...,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,-0.98%,-0.38%,-0.50%,0.00%


In [51]:
# 行列入れ替え
df_market21 = df_market2.T

In [52]:
# 1行目だけ残して後は削除
df_market22 = df_market21.drop(df_market21.index[[2,3,4]]).reset_index(drop=True)

In [53]:
# 2行目だけ残して後は削除
df_market23 = df_market21.drop(df_market21.index[[1,3,4]]).reset_index(drop=True)

In [54]:
# 3行目だけ残して後は削除
df_market24 = df_market21.drop(df_market21.index[[1,2,4]]).reset_index(drop=True)

In [55]:
df_market25 = pd.concat([df_market22, df_market23, df_market24], axis=1)

In [56]:
df_market25

,1,2,3,4,5,6,7,8,9,10,...,9,10,11,12,13,14,15,16,17,18
0,日経225先物（期近）,CME日経平均先物,SGX日経平均先物,東証マザース先物（期近）,JPX日経400先物（期近）,JPX国債先物,JPX国債先物レバレッジ,JPX国債先物インバース,JPX国債先物ダブルインバース,台湾加権先物（期近）,...,JPX国債先物ダブルインバース,台湾加権先物（期近）,FTSE中国50先物（期近）,E-mini S&P 500,E-mini S&P Mid Cap 400,E-mini S&P Small Cap 600,E-mini NASDAQ-100,E-mini Dow,E-mini Nikkei225,SGX Nifty 50 Index Futures
1,"26,920","26,930","26,935.00",750.0,"17,020","11,716.01","13,645.21","8,484.78","7,156.54",0,...,0.09%,--%,--%,0.27%,0.31%,--%,0.43%,0.22%,--%,0.16%


In [57]:
# 行列入れ替え
df_market31 = df_market3.T

In [58]:
# 1行目だけ残して後は削除
df_market32 = df_market31.drop(df_market31.index[[2,3,4,5]]).reset_index(drop=True)

In [59]:
# 2行目だけ残して後は削除
df_market33 = df_market31.drop(df_market31.index[[1,3,4,5]]).reset_index(drop=True)

In [60]:
# 3行目だけ残して後は削除
df_market34 = df_market31.drop(df_market31.index[[1,2,4,5]]).reset_index(drop=True)

In [61]:
# 4行目だけ残して後は削除
df_market35 = df_market31.drop(df_market31.index[[1,2,3,5]]).reset_index(drop=True)

In [62]:
df_market36 = pd.concat([df_market32, df_market33, df_market34, df_market35], axis=1)

In [63]:
df_market36

,1,2,3,4,5,6,7,8,9,10,...,11,12,13,14,15,16,17,18,19,20
0,米ドル/円,ユーロ/円,豪ドル/円,NZドル/円,カナダドル/円,英ポンド/円,スイスフラン/円,香港ドル/円,トルコリラ/円,南アランド/円,...,ロシアルーブル/円,韓国ウォン/円,メキシコペソ/円,ブラジルレアル/円,インドルピー/円,中国人民元/円,シンガポールドル/円,タイバーツ/円,マレーシアリンギット/円,インドネシアルピア/円
1,127.19,137.03,93.47,85.56,100.86,165.35,134.54,16.2157,8.6797,8.6642,...,0.16%,0.19%,0.23%,0.18%,0.17%,0.16%,0.13%,0.10%,0.16%,0.16%


In [64]:
# 行列入れ替え
df_market41 = df_market4.T

In [65]:
# 1行目だけ残して後は削除
df_market42 = df_market41.drop(df_market41.index[[2,3,4]]).reset_index(drop=True)

In [66]:
# 2行目だけ残して後は削除
df_market43 = df_market41.drop(df_market41.index[[1,3,4]]).reset_index(drop=True)

In [67]:
# 3行目だけ残して後は削除
df_market44 = df_market41.drop(df_market41.index[[1,2,4]]).reset_index(drop=True)

In [68]:
df_market45 = pd.concat([df_market42, df_market43, df_market44], axis=1)

In [69]:
pd.set_option('display.max_rows', None) 
# defult:60,制限なし:None
pd.set_option('display.max_columns', None) 
# defult:20,制限なし:None
df_market45

,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10
0,原油（WTI原油先物）,天然ガス（Henry Hub）先物（NYMEX）,金（現物 1oz.あたり）,Gold先物（COMEX),プラチナ先物（NYMEX）,Silver先物（COMEX),Copper先物（COMEX),Corn先物（CBOT）,Wheat先物（CBOT）,Soybeans先物（CBOT）,原油（WTI原油先物）,天然ガス（Henry Hub）先物（NYMEX）,金（現物 1oz.あたり）,Gold先物（COMEX),プラチナ先物（NYMEX）,Silver先物（COMEX),Copper先物（COMEX),Corn先物（CBOT）,Wheat先物（CBOT）,Soybeans先物（CBOT）,原油（WTI原油先物）,天然ガス（Henry Hub）先物（NYMEX）,金（現物 1oz.あたり）,Gold先物（COMEX),プラチナ先物（NYMEX）,Silver先物（COMEX),Copper先物（COMEX),Corn先物（CBOT）,Wheat先物（CBOT）,Soybeans先物（CBOT）
1,107.59,7.84,"1,976.4100","1,982.9","1,015.4",26.14,4.793,813.25,"1,120.5","1,714.75",+0.64,+0.02,-2.0867,--,--,--,-0.003,--,--,--,0.60%,0.26%,-0.11%,--%,--%,--%,-0.06%,--%,--%,--%


In [70]:
# 行列入れ替え
df_market51 = df_market5.T

In [71]:
# df_market51 = df_market51.reset_index
df_market52 = df_market51.drop(df_market51.index[[2,3]]).reset_index(drop=True)

In [72]:
df_market53 = df_market51.drop(df_market51.index[[1,3]]).reset_index(drop=True)

In [73]:
df_market54 = pd.concat([df_market52, df_market53], axis=1)

In [74]:
df_market54

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,日本国債3年,日本国債5年,日本国債10年,米国3年国債,米国10年国債,ドイツ10年国債,イギリス10年国債,フランス10年国債,スペイン10年国債,ポルトガル10年国債,ユーロ圏10年国債,イタリア10年国債,ギリシャ10年国債,ハンガリー10年国債,日本国債3年,日本国債5年,日本国債10年,米国3年国債,米国10年国債,ドイツ10年国債,イギリス10年国債,フランス10年国債,スペイン10年国債,ポルトガル10年国債,ユーロ圏10年国債,イタリア10年国債,ギリシャ10年国債,ハンガリー10年国債
1,-0.030,0.025,0.243,2.669,2.857,0.841,1.884,1.336,1.792,1.856,0.841,2.523,2.924,6.670,0.000,0.000,+0.002,+0.003,+0.049,+0.001,-0.006,-0.008,+0.004,+0.005,+0.001,+0.030,+0.036,-0.110


In [75]:
df_market61 = df_market6.T

In [76]:
df_market62 = df_market61.drop(df_market61.index[2]).reset_index(drop=True)

In [77]:
df_market62

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,日本 無担保コール翌日物,日本 公定歩合,アメリカ フェデラルファンド金利,カナダ 翌日物金利誘導目標,ユーロ 市場調整金利,イギリス レポ金利,スイス 3ヵ月物LIBOR誘導目標,南アフリカ レポ金利,香港 再割引金利,オーストラリア 政策金利,ニュージーランド 目標短期金利,インド レポ金利,中国 貸出金利,ブラジル Selicターゲットレート,ロシア リファイナンス金利,トルコ 中央銀行ON DEPO,メキシコ オーバーナイト金利
1,-0.019,0.3,0.50,1.00,--,0.75,-0.7500,4.25,5.000,0.10,1.50,4.00,4.35,11.75,--,12.50,6.48


In [78]:
df_marketall = pd.concat([df_market15, df_market25, df_market36, df_market45, df_market54, df_market62], axis=1)

In [79]:
#行と列の数を取り出す
sh=df_marketall.shape
#shapeの列の数だけrangeで振り直してcolumnsに代入
df_marketall.columns=range(sh[1])

In [80]:
# ,inplace=True
df_marketall.drop(columns=df_marketall.columns[[3,4,5,6,7,34,35,36,37,38,
                                               65,66,67,68,69]],inplace=True)

In [81]:
df_marketall

,0,1,2,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301
0,日経225,TOPIX,JPX日経400,マザーズ総合指数,日経平均VI指数,東証REIT指数,NYダウ,NASDAQ,S&P500指数,英FT100指数,仏CAC40指数,独DAX30指数,ロシアRTS指数,ブラジルボベスパ指数,インドSENSEX,香港ハンセン指数,ハンセンH株指数,ハンセンレッドチップ指数,上海総合指数,上海A株指数,上海・香港ストックコネクト,CSI300,シンガポールST指数,タイSET指数,クアラルンプール総合指数,ジャカルタ総合指数,日経225,TOPIX,JPX日経400,マザーズ総合指数,日経平均VI指数,東証REIT指数,NYダウ,NASDAQ,S&P500指数,英FT100指数,仏CAC40指数,独DAX30指数,ロシアRTS指数,ブラジルボベスパ指数,インドSENSEX,香港ハンセン指数,ハンセンH株指数,ハンセンレッドチップ指数,上海総合指数,上海A株指数,上海・香港ストックコネクト,CSI300,シンガポールST指数,タイSET指数,クアラルンプール総合指数,ジャカルタ総合指数,日経225,TOPIX,JPX日経400,マザーズ総合指数,日経平均VI指数,東証REIT指数,NYダウ,NASDAQ,S&P500指数,英FT100指数,仏CAC40指数,独DAX30指数,ロシアRTS指数,ブラジルボベスパ指数,インドSENSEX,香港ハンセン指数,ハンセンH株指数,ハンセンレッドチップ指数,上海総合指数,上海A株指数,上海・香港ストックコネクト,CSI300,シンガポールST指数,タイSET指数,クアラルンプール総合指数,ジャカルタ総合指数,日経225先物（期近）,CME日経平均先物,SGX日経平均先物,東証マザース先物（期近）,JPX日経400先物（期近）,JPX国債先物,JPX国債先物レバレッジ,JPX国債先物インバース,JPX国債先物ダブルインバース,台湾加権先物（期近）,FTSE中国50先物（期近）,E-mini S&P 500,E-mini S&P Mid Cap 400,E-mini S&P Small Cap 600,E-mini NASDAQ-100,E-mini Dow,E-mini Nikkei225,SGX Nifty 50 Index Futures,日経225先物（期近）,CME日経平均先物,SGX日経平均先物,東証マザース先物（期近）,JPX日経400先物（期近）,JPX国債先物,JPX国債先物レバレッジ,JPX国債先物インバース,JPX国債先物ダブルインバース,台湾加権先物（期近）,FTSE中国50先物（期近）,E-mini S&P 500,E-mini S&P Mid Cap 400,E-mini S&P Small Cap 600,E-mini NASDAQ-100,E-mini Dow,E-mini Nikkei225,SGX Nifty 50 Index Futures,日経225先物（期近）,CME日経平均先物,SGX日経平均先物,東証マザース先物（期近）,JPX日経400先物（期近）,JPX国債先物,JPX国債先物レバレッジ,JPX国債先物インバース,JPX国債先物ダブルインバース,台湾加権先物（期近）,FTSE中国50先物（期近）,E-mini S&P 500,E-mini S&P Mid Cap 400,E-mini S&P Small Cap 600,E-mini NASDAQ-100,E-mini Dow,E-mini Nikkei225,SGX Nifty 50 Index Futures,米ドル/円,ユーロ/円,豪ドル/円,NZドル/円,カナダドル/円,英ポンド/円,スイスフラン/円,香港ドル/円,トルコリラ/円,南アランド/円,ロシアルーブル/円,韓国ウォン/円,メキシコペソ/円,ブラジルレアル/円,インドルピー/円,中国人民元/円,シンガポールドル/円,タイバーツ/円,マレーシアリンギット/円,インドネシアルピア/円,米ドル/円,ユーロ/円,豪ドル/円,NZドル/円,カナダドル/円,英ポンド/円,スイスフラン/円,香港ドル/円,トルコリラ/円,南アランド/円,ロシアルーブル/円,韓国ウォン/円,メキシコペソ/円,ブラジルレアル/円,インドルピー/円,中国人民元/円,シンガポールドル/円,タイバーツ/円,マレーシアリンギット/円,インドネシアルピア/円,米ドル/円,ユーロ/円,豪ドル/円,NZドル/円,カナダドル/円,英ポンド/円,スイスフラン/円,香港ドル/円,トルコリラ/円,南アランド/円,ロシアルーブル/円,韓国ウォン/円,メキシコペソ/円,ブラジルレアル/円,インドルピー/円,中国人民元/円,シンガポールドル/円,タイバーツ/円,マレーシアリンギット/円,インドネシアルピア/円,米ドル/円,ユーロ/円,豪ドル/円,NZドル/円,カナダドル/円,英ポンド/円,スイスフラン/円,香港ドル/円,トルコリラ/円,南アランド/円,ロシアルーブル/円,韓国ウォン/円,メキシコペソ/円,ブラジルレアル/円,インドルピー/円,中国人民元/円,シンガポールドル/円,タイバーツ/円,マレーシアリンギット/円,インドネシアルピア/円,原油（WTI原油先物）,天然ガス（Henry Hub）先物（NYMEX）,金（現物 1oz.あたり）,Gold先物（COMEX),プラチナ先物（NYMEX）,Silver先物（COMEX),Copper先物（COMEX),Corn先物（CBOT）,Wheat先物（CBOT）,Soybeans先物（CBOT）,原油（WTI原油先物）,天然ガス（Henry Hub）先物（NYMEX）,金（現物 1oz.あたり）,Gold先物（COMEX),プラチナ先物（NYMEX）,Silver先物（COMEX),Copper先物（COMEX),Corn先物（CBOT）,Wheat先物（CBOT）,Soybeans先物（CBOT）,原油（WTI原油先物）,天然ガス（Henry Hub）先物（NYMEX）,金（現物 1oz.あたり）,Gold先物（COMEX),プラチナ先物（NYMEX）,Silver先物（COMEX),Copper先物（COMEX),Corn先物（CBOT）,Wheat先物（CBOT）,Soybeans先物（CBOT）,日本国債3年,日本国債5年,日本国債10年,米国3年国債,米国10年国債,ドイツ10年国債,イギリス10年国債,フランス10年国債,スペイン10年国債,ポルトガル10年国債,ユーロ圏10年国債,イタリア10年国債,ギリシャ10年国債,ハンガリー10年国債,日本国債3年,日本国債5年,日本国債10年,米国3年国債,米国10年国債,ドイツ10年国債,イギリス10年国債,フランス10年国債,スペイン10年国債,ポルトガル10年国債,ユーロ圏10年国債,イタリア10年国債,ギリシャ10年国債,ハンガリー10年国債,日本 無担保コール翌日物,日本 公定歩合

In [82]:
# プラウザを閉じる
driver.quit()

In [83]:
df_marketall.to_csv('data/market_' + filedate + '.csv')